In [1]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset
from pjs.dataset import Data
import pandas as pd
import torch
from loguru import logger

c:\Users\laure\AppData\Local\pypoetry\Cache\virtualenvs\pjs-8TwGqzJm-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = Data(data_path="../data/LMentry")

2025-04-18 12:55:46.934 | INFO     | pjs.dataset:collect_data:41 - Successfully loaded 'examples' from: ..\data\LMentry\bigger_number.json
2025-04-18 12:55:46.942 | INFO     | pjs.dataset:collect_data:41 - Successfully loaded 'examples' from: ..\data\LMentry\first_alphabetically.json
2025-04-18 12:55:46.948 | INFO     | pjs.dataset:collect_data:41 - Successfully loaded 'examples' from: ..\data\LMentry\first_alphabetically_consecutive_first_letter.json
2025-04-18 12:55:46.954 | INFO     | pjs.dataset:collect_data:41 - Successfully loaded 'examples' from: ..\data\LMentry\first_alphabetically_different_first_letter.json
2025-04-18 12:55:46.961 | INFO     | pjs.dataset:collect_data:41 - Successfully loaded 'examples' from: ..\data\LMentry\first_alphabetically_far_first_letter.json
2025-04-18 12:55:46.967 | INFO     | pjs.dataset:collect_data:41 - Successfully loaded 'examples' from: ..\data\LMentry\first_alphabetically_same_first_letter.json
2025-04-18 12:55:46.972 | INFO     | pjs.dataset

In [3]:
data.load_split("../data/dataset_splits/LMentry_split")

2025-04-18 12:55:47.197 | INFO     | pjs.dataset:load_split:121 - loaded: bigger_number.json
2025-04-18 12:55:47.199 | INFO     | pjs.dataset:load_split:121 - loaded: bigger_number.json
2025-04-18 12:55:47.200 | INFO     | pjs.dataset:load_split:121 - loaded: first_alphabetically_consecutive_first_letter.json
2025-04-18 12:55:47.203 | INFO     | pjs.dataset:load_split:121 - loaded: first_alphabetically_consecutive_first_letter.json
2025-04-18 12:55:47.204 | INFO     | pjs.dataset:load_split:121 - loaded: first_alphabetically_different_first_letter.json
2025-04-18 12:55:47.206 | INFO     | pjs.dataset:load_split:121 - loaded: first_alphabetically_different_first_letter.json
2025-04-18 12:55:47.207 | INFO     | pjs.dataset:load_split:121 - loaded: first_alphabetically_far_first_letter.json
2025-04-18 12:55:47.209 | INFO     | pjs.dataset:load_split:121 - loaded: first_alphabetically_far_first_letter.json
2025-04-18 12:55:47.210 | INFO     | pjs.dataset:load_split:121 - loaded: first_alph

In [4]:
model_path = "../data/models/task_data_train/bigger_number.json/checkpoint-900"

## Macht W's statt leerzeichen, vielleicht lieber BABYLM-TOKENIZER-CHAR-TXT-SPACELESS ?
tokenizer = AutoTokenizer.from_pretrained('phonemetransformers/babble-tokenizers', subfolder='BABYLM-TOKENIZER-CHAR-TXT')

model = GPT2LMHeadModel.from_pretrained(model_path)

In [5]:
input_text = "Q: In an alphabetical order, which word comes first, \"blanket\" or \"diary\"? A:"
tokens = tokenizer.tokenize(input_text)
logger.info(tokens)

2025-04-18 12:55:47.531 | INFO     | __main__:<module>:3 - ['q', ':', 'W', 'i', 'n', 'W', 'a', 'n', 'W', 'a', 'l', 'p', 'h', 'a', 'b', 'e', 't', 'i', 'c', 'a', 'l', 'W', 'o', 'r', 'd', 'e', 'r', ',', 'W', 'w', 'h', 'i', 'c', 'h', 'W', 'w', 'o', 'r', 'd', 'W', 'c', 'o', 'm', 'e', 's', 'W', 'f', 'i', 'r', 's', 't', ',', 'W', '"', 'b', 'l', 'a', 'n', 'k', 'e', 't', '"', 'W', 'o', 'r', 'W', '"', 'd', 'i', 'a', 'r', 'y', '"', '?', 'W', 'a', ':']


In [10]:
split_data = data.split_data
test_df = split_data["bigger_number.json"]["test"]

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(115, 768)
    (wpe): Embedding(256, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=115, bias=False)
)

In [ ]:
from pjs.test import test_model


test_model(model, tokenizer, test_df, "../data/outputs/", "bigger_number.json")


Input: Q: Of the numbers 701 and 438, which is bigger? A:
Expected: 701
Predicted: 701, which is bigger? 
---
Input: Q: Of the numbers 486 and 491, which is bigger? A:
Expected: 491
Predicted: 491, which is bigger? 
---
